In [136]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

In [137]:
import cv2
import tensorflow as tf
import numpy as np
import csv
from PIL import Image

In [138]:
width=256
height=256
depth=3

inputShape=(width,height,depth)

In [139]:
model = Sequential()

model.add(Conv2D(8, (3, 3),input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.1))

model.add(Conv2D(16, (3, 3)))
model.add(Activation("relu"))

model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))

model.add(Conv2D(128, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

model.add(Flatten())

model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.1))

model.add(Dense(units=2,activation='softmax'))


In [140]:
epochs = 10
learning_rate = 0.001
batch_size = 8

opt = Adam(lr=learning_rate, decay=learning_rate / epochs)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=["sparse_categorical_accuracy"])

In [141]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_67 (Conv2D)           (None, 254, 254, 8)       224       
_________________________________________________________________
activation_78 (Activation)   (None, 254, 254, 8)       0         
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 84, 84, 8)         0         
_________________________________________________________________
dropout_56 (Dropout)         (None, 84, 84, 8)         0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 82, 82, 16)        1168      
_________________________________________________________________
activation_79 (Activation)   (None, 82, 82, 16)        0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 80, 80, 32)      

In [159]:
#model.load_weights('bumps.h5')
for i in range(epochs):
    print('Epoch:',i)
    
    X_train=list()
    y_train=list()
    with open('./Train_label.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        cnt=1
        for row in csv_reader:
            if(cnt%500==0):
                X_train=np.asarray(X_train)
                X_train = X_train.reshape(-1,256, 256,3)
                X_train=np.array(X_train)   
                y_train=np.array(y_train)           
                model.fit(X_train,y_train,batch_size=batch_size)
                X_train=list()
                y_train=list()
                pil_im = Image.open(row[1], 'r')
                #imshow(np.asarray(pil_im))
                a=np.asarray(pil_im,dtype="float32" )

                a=a/255.0
                X_train.append(a)
                b=row[2]
                y_train.append(int(b))
                cnt+=1
            else:
                #print(row[1],row[2])
                pil_im = Image.open(row[1], 'r')
                #imshow(np.asarray(pil_im))
                a=np.asarray(pil_im,dtype="float32" )

                a=a/255.0
                # print(a.shape,row[1],row[2])
                X_train.append(a)
                b=row[2]
                y_train.append(int(b))
                cnt+=1


    X_train=np.asarray(X_train)
    print(X_train.shape)
    X_train = X_train.reshape(-1,256, 256,3)
    X_train=np.array(X_train)   
    y_train=np.array(y_train)            
    model.fit(X_train,y_train)
    model.save('bumps.h5')

Epoch: 0
Epoch 1/1
499/499 [==============================] - 18s 37ms/step - loss: 0.7191 - sparse_categorical_accuracy: 0.6533
(240, 256, 256, 3)
Epoch 1/1
240/240 [==============================] - 5s 23ms/step - loss: 0.5324 - sparse_categorical_accuracy: 0.8125
Epoch: 1
Epoch 1/1
499/499 [==============================] - 18s 37ms/step - loss: 0.5005 - sparse_categorical_accuracy: 0.7816
(240, 256, 256, 3)
Epoch 1/1
240/240 [==============================] - 5s 23ms/step - loss: 0.3074 - sparse_categorical_accuracy: 0.8833
Epoch: 2
Epoch 1/1
499/499 [==============================] - 19s 37ms/step - loss: 0.3849 - sparse_categorical_accuracy: 0.8477
(240, 256, 256, 3)
Epoch 1/1
240/240 [==============================] - 5s 23ms/step - loss: 0.2951 - sparse_categorical_accuracy: 0.8875
Epoch: 3
Epoch 1/1
499/499 [==============================] - 17s 35ms/step - loss: 0.3672 - sparse_categorical_accuracy: 0.8717
(240, 256, 256, 3)
Epoch 1/1
240/240 [==============================] 

In [143]:
model.save('bumps.h5')

In [144]:
model.load_weights('bumps.h5')

In [145]:
predict_list=[]
actual_list=[]
correct=0
total=0
with open('./Test_label.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        
        y_test=row[2]
        #X_test=list()
        pil_im = Image.open(row[1], 'r')
        X_test=np.asarray(pil_im,dtype="float" )
        X_test=X_test/255.0
        #X_test.append(a)
        #X_test=np.asarray(X_test)
        X_test = X_test.reshape(-1,256, 256,3)
        prediction = list(model.predict(X_test))
#         print('Prediction|Actual')
#         print(np.argmax(prediction[0]),row[2])
#         y_test=row[2]
        predict_list.append(np.argmax(prediction[0]))
        actual_list.append()
        if(np.argmax(prediction[0])==int(y_test)):
            correct+=1
            total+=1
        else:
            total+=1
        
        #print(prediction[0][0],prediction[0][1],prediction[0][2],prediction[0][3],row[2])
        cur_acc=(correct/total)*100.0
        print('Current accuracy is:{0}%'.format(cur_acc))
acc=(correct/total)*100.0
print('Sparse-categorical-accuracy(Final) is: {0}%'.format(acc))

TypeError: append() takes exactly one argument (0 given)

In [146]:
from sklearn.metrics import confusion_matrix

In [147]:
confusion_matrix(actual_list,predict_list)

ValueError: Found input variables with inconsistent numbers of samples: [0, 1]